In [13]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow.contrib.layers import fully_connected


mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# X转为784列的矩阵
# X_train = X_train.reshape((-1, 784))
# X_test = X_test.reshape((-1, 784))

X_train = X_train.reshape((-1, 28, 28 , 1))
X_test = X_test.reshape((-1, 28, 28 , 1))

''' 这里是个坑，如果不进行数据归一化，得到的准确率非常低'''
X_train = X_train / 255.0
X_test = X_test / 255.0


# y值转为oneHot编码
one_hot_encoder = OneHotEncoder(sparse=False)
y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(len(y_train), 1))
y_test_one_hot = one_hot_encoder.fit_transform(y_test.reshape(len(y_test), 1))

# 根据传入的数据格式生成W初始值
def weight_variable(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)

# 初始化b
def bias_variable(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

# 定义卷积函数
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

# 定义池化函数
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding="SAME")

x = tf.placeholder(tf.float32, [None, 28, 28 , 1])
y_ = tf.placeholder(tf.float32, [None, 10])


# 第一层CNN 
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 第二层CNN
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 第一层全连接
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout层，防止过拟合
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# SoftMax分类
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# 定义损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 开始训练
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

batch_size = 50
for i in range(len(X_train // batch_size)):
    batch_x = X_train[i*batch_size:min(i*batch_size+batch_size, len(X_train))]
    batch_y = y_train_one_hot[i*batch_size:min(i*batch_size+batch_size, len(X_train))]
    
    if i % 100 == 0:
        train_acc = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print("train acc", train_acc)
        
        test_acc = accuracy.eval(feed_dict={x:X_test, y_:y_test_one_hot, keep_prob:1.0})
        print("test acc", test_acc)
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})
    
    

' 这里是个坑，如果不进行数据归一化，得到的准确率非常低'

/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


train acc 0.14
test acc 0.1031
train acc 0.88
test acc 0.8431
train acc 0.9
test acc 0.8869
train acc 0.92
test acc 0.9238
train acc 0.92
test acc 0.9337
train acc 1.0
test acc 0.9407
train acc 0.92
test acc 0.9502
train acc 1.0
test acc 0.9574
train acc 0.98
test acc 0.9524
train acc 0.9
test acc 0.9618
train acc 1.0
test acc 0.9632
train acc 1.0
test acc 0.9652


InternalError: Missing 1-th output from node MaxPool_19 (defined at <ipython-input-13-ecd1bf7faf37>:44) 